## Cleaning Dataframe

In [1]:
import pandas as pd

In [23]:
data = pd.read_csv('../../data/processed/total.csv')

In [27]:
data.head()

,year,age,attempts,yards_run,tds_run,longgain_run,yardsperatt,yardspergame_run,Percenthit (%),g,...,catchpercent,yards_rec,yardsperrec,tds_rec,firstdowns,longgain_rec,yardspertarget,recpergame,yardspergame_rec,fumbles
0,2020,26,75,389,5,75,5.2,77.8,1.100909,9,...,73.3%,261,7.9,2,12,30,5.8,3.7,29.0,2.0
1,2019,25,236,1084,16,56,4.6,67.8,0.369547,16,...,72.1%,474,9.7,3,18,67,7.0,3.1,29.6,3.0
2,2018,24,133,728,8,67,5.5,60.7,0.293098,12,...,74.3%,206,7.9,1,12,24,5.9,2.2,17.2,1.0
3,2017,23,81,448,4,46,5.5,37.3,0.308675,12,...,50.0%,22,2.4,0,2,9,1.2,0.8,1.8,0.0
4,2020,35,80,314,2,27,3.9,52.3,0.529768,11,...,68.8%,78,7.1,0,3,18,4.9,1.0,7.1,0.0


In [29]:
data.columns

Index(['year', 'age', 'attempts', 'yards_run', 'tds_run', 'longgain_run',
       'yardsperatt', 'yardspergame_run', 'Percenthit (%)', 'g', 'gs', 'tgt',
       'rec', 'catchpercent', 'yards_rec', 'yardsperrec', 'tds_rec',
       'firstdowns', 'longgain_rec', 'yardspertarget', 'recpergame',
       'yardspergame_rec', 'fumbles'],
      dtype='object')

In [25]:
data = data.drop('Unnamed: 0',axis=1)
data = data.drop('playername', axis=1)
data = data.drop('team', axis=1)

In [26]:
data = data.drop('basesalarycap (10^8)',axis=1)
data = data.drop('caphit (M)',axis=1)
data = data.drop('cashspent (M)',axis=1)

In [30]:
data = data.drop('catchpercent',axis=1)

In [31]:
y_val = data['Percenthit (%)']

In [32]:
x_data = data.drop('Percenthit (%)',axis=1)

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3)

## Scaling data

In [35]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
scaler = MinMaxScaler()

In [37]:
scaler.fit(x_train)

/home/brian/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [38]:
x_train = pd.DataFrame(data=scaler.transform(x_train),
                       columns=x_train.columns,index=x_train.index)
x_test = pd.DataFrame(data=scaler.transform(x_test),
                      columns=x_test.columns,index=x_test.index)

## Creating Featuring Columns

In [39]:
data.columns

Index(['year', 'age', 'attempts', 'yards_run', 'tds_run', 'longgain_run',
       'yardsperatt', 'yardspergame_run', 'Percenthit (%)', 'g', 'gs', 'tgt',
       'rec', 'yards_rec', 'yardsperrec', 'tds_rec', 'firstdowns',
       'longgain_rec', 'yardspertarget', 'recpergame', 'yardspergame_rec',
       'fumbles'],
      dtype='object')

In [40]:
import tensorflow as tf

In [41]:
year = tf.feature_column.numeric_column('year')
age = tf.feature_column.numeric_column('age')
attempts = tf.feature_column.numeric_column('attempts')
runyards = tf.feature_column.numeric_column('yards_run')
runtds = tf.feature_column.numeric_column('tds_run')
lgrun = tf.feature_column.numeric_column('longgain_run')
ypa = tf.feature_column.numeric_column('yardsperatt')
ypgrun = tf.feature_column.numeric_column('yardspergame_run')
games = tf.feature_column.numeric_column('g')
started = tf.feature_column.numeric_column('gs')
targets = tf.feature_column.numeric_column('tgt')
receptions = tf.feature_column.numeric_column('rec')
recyards = tf.feature_column.numeric_column('yards_rec')
ypr = tf.feature_column.numeric_column('yardsperrec')
rectds = tf.feature_column.numeric_column('tds_rec')
firstdown = tf.feature_column.numeric_column('firstdowns')
lgrec = tf.feature_column.numeric_column('longgain_rec')
ypt = tf.feature_column.numeric_column('yardspertarget')
rpg = tf.feature_column.numeric_column('recpergame')
ypgrec = tf.feature_column.numeric_column('yardspergame_rec')
fum = tf.feature_column.numeric_column('fumbles')

In [42]:
feat_cols = [year,age,attempts,runyards,runtds,lgrun,ypa,ypgrun,games,started,
            targets,receptions,recyards,ypr,rectds,firstdown,lgrec,ypt,rpg,ypgrec,fum]

In [43]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                                batch_size=10,
                                                num_epochs=1000,
                                                shuffle=True)

In [44]:
model = tf.estimator.DNNRegressor(hidden_units=[12,12,12],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_session_config': None, '_tf_random_seed': 1, '_model_dir': '/tmp/tmpknry0ldc', '_keep_checkpoint_every_n_hours': 10000}


In [46]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpknry0ldc/model.ckpt.
INFO:tensorflow:step = 1, loss = 35.063457
INFO:tensorflow:global_step/sec: 660.647
INFO:tensorflow:step = 101, loss = 33.838688 (0.154 sec)
INFO:tensorflow:global_step/sec: 689.791
INFO:tensorflow:step = 201, loss = 27.953728 (0.145 sec)
INFO:tensorflow:global_step/sec: 709.556
INFO:tensorflow:step = 301, loss = 12.86631 (0.141 sec)
INFO:tensorflow:global_step/sec: 603.041
INFO:tensorflow:step = 401, loss = 6.8085775 (0.166 sec)
INFO:tensorflow:global_step/sec: 742.029
INFO:tensorflow:step = 501, loss = 14.097652 (0.136 sec)
INFO:tensorflow:global_step/sec: 660.845
INFO:tensorflow:step = 601, loss = 51.53769 (0.150 sec)
INFO:tensorflow:global_step/sec: 672.368
INFO:tensorflow:step = 701, loss = 20.821526 (0.149 sec)
INFO:tensorflow:global_step/sec: 621.07
INFO:tensorflow:step = 801, loss = 53.62687 (0.161 sec)
INFO:tensorflow:global_step/sec: 634.291
INFO:tensorflow:s

INFO:tensorflow:global_step/sec: 747.492
INFO:tensorflow:step = 8301, loss = 19.486795 (0.134 sec)
INFO:tensorflow:global_step/sec: 756.321
INFO:tensorflow:step = 8401, loss = 24.074867 (0.133 sec)
INFO:tensorflow:global_step/sec: 600.339
INFO:tensorflow:step = 8501, loss = 13.1052885 (0.166 sec)
INFO:tensorflow:global_step/sec: 522.685
INFO:tensorflow:step = 8601, loss = 16.486118 (0.191 sec)
INFO:tensorflow:global_step/sec: 604.743
INFO:tensorflow:step = 8701, loss = 2.175566 (0.165 sec)
INFO:tensorflow:global_step/sec: 682.706
INFO:tensorflow:step = 8801, loss = 8.589292 (0.146 sec)
INFO:tensorflow:global_step/sec: 754.52
INFO:tensorflow:step = 8901, loss = 19.454569 (0.134 sec)
INFO:tensorflow:global_step/sec: 752.61
INFO:tensorflow:step = 9001, loss = 8.515396 (0.131 sec)
INFO:tensorflow:global_step/sec: 702.569
INFO:tensorflow:step = 9101, loss = 13.012854 (0.142 sec)
INFO:tensorflow:global_step/sec: 709.947
INFO:tensorflow:step = 9201, loss = 4.887786 (0.142 sec)
INFO:tensorflow

In [47]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,
                                                        batch_size=10,
                                                        num_epochs=1,
                                                        shuffle=False)

In [48]:
pred_gen = model.predict(predict_input_func)

In [49]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmpknry0ldc/model.ckpt-10000


In [50]:
predictions

[{'predictions': array([3.2678058], dtype=float32)},
 {'predictions': array([0.300961], dtype=float32)},
 {'predictions': array([2.803298], dtype=float32)},
 {'predictions': array([0.52514297], dtype=float32)},
 {'predictions': array([1.3622879], dtype=float32)},
 {'predictions': array([1.2586339], dtype=float32)},
 {'predictions': array([0.70978945], dtype=float32)},
 {'predictions': array([1.257666], dtype=float32)},
 {'predictions': array([0.86717415], dtype=float32)},
 {'predictions': array([0.43171293], dtype=float32)},
 {'predictions': array([0.43171293], dtype=float32)},
 {'predictions': array([2.49251], dtype=float32)},
 {'predictions': array([2.721813], dtype=float32)},
 {'predictions': array([0.6113038], dtype=float32)},
 {'predictions': array([4.798425], dtype=float32)},
 {'predictions': array([0.438726], dtype=float32)},
 {'predictions': array([0.51776177], dtype=float32)},
 {'predictions': array([0.4538188], dtype=float32)},
 {'predictions': array([0.97536373], dtype=float

In [51]:
final_preds = []

for pred in predictions:
    final_preds.append(pred['predictions'])

In [52]:
from sklearn.metrics import mean_squared_error

In [53]:
mean_squared_error(y_test,final_preds)**0.5

1.568759231443841

In [56]:
final_preds

[array([3.2678058], dtype=float32),
 array([0.300961], dtype=float32),
 array([2.803298], dtype=float32),
 array([0.52514297], dtype=float32),
 array([1.3622879], dtype=float32),
 array([1.2586339], dtype=float32),
 array([0.70978945], dtype=float32),
 array([1.257666], dtype=float32),
 array([0.86717415], dtype=float32),
 array([0.43171293], dtype=float32),
 array([0.43171293], dtype=float32),
 array([2.49251], dtype=float32),
 array([2.721813], dtype=float32),
 array([0.6113038], dtype=float32),
 array([4.798425], dtype=float32),
 array([0.438726], dtype=float32),
 array([0.51776177], dtype=float32),
 array([0.4538188], dtype=float32),
 array([0.97536373], dtype=float32),
 array([1.1730336], dtype=float32),
 array([1.6040019], dtype=float32),
 array([1.273647], dtype=float32),
 array([0.67465895], dtype=float32),
 array([0.4886074], dtype=float32),
 array([1.2383277], dtype=float32),
 array([4.2152357], dtype=float32),
 array([5.261278], dtype=float32),
 array([0.809962], dtype=float

In [70]:
type(y_test)

pandas.core.series.Series

### looking through the y_test and the final_preds, some of the percentages were very close, some were very off. The root mean square error was 1.5% percent hit, which is closer from the 3% percent hit we had earlier.

In [63]:
len(final_preds)

181

In [71]:
pred_total = 0
test_total = 0

for pred in final_preds:
    pred_total = pred_total + pred[0]
    
for test in y_test.values:
    test_total = test_total + test

In [88]:
y_test_values = y_test.values
total_error = 0
percent_error = 0

for i in range(0,len(final_preds)):
    true_error = final_preds[i][0] - y_test_values[i] 
    print(true_error)
    total_error = total_error + true_error
    percent_error = percent_error + ((true_error /
                                     y_test_values[i]) / len(final_preds))



0.4225219546978951
0.0663418261296734
2.1439988095291262
-0.16103261252103962
-0.3741704543431599
1.1505435391896668
0.20706079996622095
0.930785819369349
-0.5260256257067952
0.02342146814768037
-1.212585762759631
-1.956110972294053
2.095771296819051
-2.0357550172244805
-0.17669918050813482
0.15928528937551079
0.20361616871642207
0.026318100131070443
0.5851198289452529
-1.9405281696207384
-0.8296888652289156
-0.9838129295177804
-0.49431022757804866
0.016370280867707976
0.8687809828556559
3.859705236103411
3.17076091108651
-1.2127423644306439
-0.17305249523224864
0.04702446555638695
-0.8725923855930531
0.17528238058039536
0.08322896895301074
4.678555357045141
1.7445334008069542
-0.11659287368922894
-2.3428435958384677
0.24838822837767616
-0.16221775997012855
-0.7758138512237043
0.0018175545718278063
0.4204888391225299
-0.6947077322955164
-5.722533916998462
0.5220172942018696
-0.5275133775589163
-0.11162628173828126
0.09420875668164819
1.7847450998546122
-0.1722626403614771
0.70483925720

In [89]:
print("combined error: ")
print(total_error)
print("combined error / # of entries: ")
print(total_error/(len(final_preds)))
print("percent error: ")
print(percent_error)

combined error: 
2.7404167220558193
combined error / # of entries: 
0.015140423878761433
percent error: 
1.0076691761426129
